# パッケージの読み込み

In [169]:
import os
# 数値処理
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, KFold, cross_validate

# グラフ描画
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Meiryo'

# 学習モデル
import optuna
import lightgbm as lgb

# データの読み込み

In [164]:
# データの配置場所
output_dir = 'C:/Users/rhira/Documents/GitHub'\
             '/signate-competition-Liver-disease-beginner/features'

# ダミー変数化のみ
train_dummied = pd.read_feather(os.path.join(output_dir, 'train_dummied_feature.feather'))
test_dummied = pd.read_feather(os.path.join(output_dir, 'test_dummied_feature.feather'))
# 2次の交互作用まで
train_quadratic = pd.read_feather(os.path.join(output_dir, 'train_qiadratic_feature.feather'))
test_quadratic = pd.read_feather(os.path.join(output_dir, 'test_quadratic_feature.feather'))
# 3次の交互作用まで
train_cubic = pd.read_feather(os.path.join(output_dir, 'train_qiadratic_feature.feather'))
test_cubic = pd.read_feather(os.path.join(output_dir, 'test_quadratic_feature.feather'))

# ターゲット
train_target = pd.read_feather(os.path.join(output_dir, 'train_target.feather'))

# optuna用の関数の作成

In [185]:
def objectives(trial):
   
    # optunaでのハイパーパラメータサーチ範囲の設定
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.05,
        'max_bin': trial.suggest_int('max_bin', 255, 500),
        'num_leaves': trial.suggest_int('num_leaves', 10, 300),
        'reg_alpha': trial.suggest_loguniform('reg_alpha',0.001, 10),
        'reg_lambda':trial.suggest_loguniform('reg_lambda', 0.001, 10)
    }
    
    lgb_train = lgb.Dataset(X_train, y_train,
                            categorical_feature=categorical_feature)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train, categorical_feature=categorical_feature)
    
    model = lgb.train(params, lgb_train,
                      valid_sets=[lgb_train, lgb_eval],
                      verbose_eval=10,
                      num_boost_round=1000,
                      early_stopping_rounds=10)

    y_pred_test = model.predict(X_test, num_iteration=model.best_iteration)
    
    score = roc_auc_score(y_test, np.rint(y_pred_test))

    return score

# ダミー変数化データの学習

In [186]:
# 使用する特徴量とターゲット
feature = train_dummied
target = train_target
# train-test split
X_train, X_test, y_train, y_test = train_test_split(feature, target, random_state=0, stratify=target)

# optunaによる最適化呼び出し
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objectives, n_trials=40)

[I 2020-09-22 13:30:38,060] A new study created in memory with name: no-name-a656f21d-b8cc-4417-976e-07ed4c2ccc3d
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[LightGBM] [Info] Number of positive: 283, number of negative: 354
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000210 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1748
[LightGBM] [Info] Number of data points in the train set: 637, number of used features: 10


C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.444270 -> initscore=-0.223850
[LightGBM] [Info] Start training from score -0.223850
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[10]	training's auc: 0.961071	valid_1's auc: 0.90281
[L

[I 2020-09-22 13:30:38,353] Trial 0 finished with value: 0.835414808206958 and parameters: {'max_bin': 427, 'num_leaves': 57, 'reg_alpha': 0.7257005721594281, 'reg_lambda': 0.25766385746135895}. Best is trial 0 with value: 0.835414808206958.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


...

[I 2020-09-22 13:30:50,593] Trial 36 finished with value: 0.8364406779661017 and parameters: {'max_bin': 376, 'num_leaves': 127, 'reg_alpha': 6.673206111956788, 'reg_lambda': 0.3766697960577614}. Best is trial 22 with value: 0.8491525423728814.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[LightGBM] [Info] Number of positive: 283, number of negative: 354
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000252 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1748
[LightGBM] [Info] Number of data points in the train set: 637, number of used features: 10


C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.444270 -> initscore=-0.223850
[LightGBM] [Info] Start training from score -0.223850
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[10]	training's auc: 0.963087	valid_1's auc: 0.915477
[

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [188]:
params = {
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.05,
        'max_bin': study.best_params['max_bin'],
        'num_leaves': study.best_params['num_leaves'],
        'reg_alpha': study.best_params['reg_alpha'],
        'reg_lambda':study.best_params['reg_lambda']
}

lgb_train = lgb.Dataset(X_train, y_train,
                                         categorical_feature=categorical_feature)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train, categorical_feature=categorical_feature)

model = lgb.train(params, lgb_train,
                               valid_sets=[lgb_train, lgb_eval],
                               verbose_eval=10,
                               num_boost_round=1000,
                               early_stopping_rounds=10)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[LightGBM] [Info] Number of positive: 283, number of negative: 354
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000347 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1748
[LightGBM] [Info] Number of data points in the train set: 637, number of used features: 10


C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.444270 -> initscore=-0.223850
[LightGBM] [Info] Start training from score -0.223850
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[10]	training's auc: 0.963207	valid_1's auc: 0.914853
[

In [192]:
y_pred = model.predict(test_dummied, num_iteration=model.best_iteration)
pd.DataFrame(np.round(y_pred)).to_csv('dummied_optuna_lightgbm.csv', header=None)

# 2次の交互作用データの学習

In [34]:
# 使用する特徴量とターゲット
feature = train_quadratic
target = train_target
# optunaによる最適化呼び出し
opt = optuna.create_study(direction='maximize')
opt.optimize(objectives, n_trials=100)


study.best_params

[I 2020-09-21 19:04:25,009] A new study created in memory with name: no-name-c2a8c9d7-2172-42da-8e6c-9d5fad5ff10e
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:04:25,756] Trial 0 finished with value: 0.8414976230548912 and parameters: {'num_leaves': 197, 'reg_alpha': 0.007457818925212517, 'reg_lambda': 0.002783503733138495}. Best is trial 0 with value: 0.8414976230548912.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:04:26,214] Trial 1 finished with value: 0.8650825287152685 and parameters: {'num_leaves': 36, 'reg_alpha': 3.143948

[I 2020-09-21 19:04:40,260] Trial 16 finished with value: 0.8414976230548912 and parameters: {'num_leaves': 35, 'reg_alpha': 0.0013163714277321084, 'reg_lambda': 5.46599698856516}. Best is trial 1 with value: 0.8650825287152685.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:04:40,709] Trial 17 finished with value: 0.8477869312309917 and parameters: {'num_leaves': 117, 'reg_alpha': 1.9461329588450311, 'reg_lambda': 0.18297082268110496}. Best is trial 1 with value: 0.8650825287152685.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:04

[I 2020-09-21 19:04:51,510] Trial 33 finished with value: 0.8430625682818081 and parameters: {'num_leaves': 205, 'reg_alpha': 1.001566408973989, 'reg_lambda': 5.438052823022747}. Best is trial 1 with value: 0.8650825287152685.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:04:52,051] Trial 34 finished with value: 0.8587784569049518 and parameters: {'num_leaves': 296, 'reg_alpha': 2.839154546638899, 'reg_lambda': 2.6103374761942066}. Best is trial 1 with value: 0.8650825287152685.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:04:52,

[I 2020-09-21 19:05:01,423] Trial 50 finished with value: 0.8525186759972835 and parameters: {'num_leaves': 212, 'reg_alpha': 2.0067545752169247, 'reg_lambda': 9.963867911121994}. Best is trial 43 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:05:01,879] Trial 51 finished with value: 0.861923110993002 and parameters: {'num_leaves': 192, 'reg_alpha': 3.171728283564597, 'reg_lambda': 1.7706900142454602}. Best is trial 43 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:05:0

[I 2020-09-21 19:05:13,257] Trial 67 finished with value: 0.8666253284908613 and parameters: {'num_leaves': 38, 'reg_alpha': 3.5890861655707598, 'reg_lambda': 9.823447792546018}. Best is trial 43 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:05:14,068] Trial 68 finished with value: 0.8446496589600496 and parameters: {'num_leaves': 32, 'reg_alpha': 1.3061345366082056, 'reg_lambda': 9.851410759778924}. Best is trial 43 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:05:14

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


[I 2020-09-21 19:05:14,972] Trial 70 finished with value: 0.8603507839489769 and parameters: {'num_leaves': 31, 'reg_alpha': 3.2476599042029584, 'reg_lambda': 0.49599746756897073}. Best is trial 43 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:05:15,661] Trial 71 finished with value: 0.8650603832639442 and parameters: {'num_leaves': 49, 'reg_alpha': 4.1138440539286405, 'reg_lambda': 6.046537056632454}. Best is trial 43 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:05:

[I 2020-09-21 19:05:24,593] Trial 87 finished with value: 0.8634880562199191 and parameters: {'num_leaves': 200, 'reg_alpha': 7.620978256678386, 'reg_lambda': 0.8500455771147704}. Best is trial 43 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:05:25,039] Trial 88 finished with value: 0.861923110993002 and parameters: {'num_leaves': 262, 'reg_alpha': 5.7683065252258094, 'reg_lambda': 1.4581429837775013}. Best is trial 43 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:05:

最終トライアル回数:100
ベストトライアル:
値:0.8666327103079694
パラメータ:
num_leaves:189
reg_alpha:3.029104937419662
reg_lambda:2.319511005423461


# 3次の交互作用データの学習

In [35]:
# 使用する特徴量とターゲット
feature = train_cubic
target = train_target
# optunaによる最適化呼び出し
opt = optuna.create_study(direction='maximize')
opt.optimize(objectives, n_trials=100)


# 実行結果表示
print('最終トライアル回数:{}'.format(len(opt.trials)))
print('ベストトライアル:')
trial = opt.best_trial
print('値:{}'.format(trial.value))
print('パラメータ:')
for key, value in trial.params.items():
    print('{}:{}'.format(key, value))

[I 2020-09-21 19:06:04,033] A new study created in memory with name: no-name-b08385f1-fee4-49cf-863c-d954bf4700de
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:04,652] Trial 0 finished with value: 0.8383751144181651 and parameters: {'num_leaves': 173, 'reg_alpha': 0.05916538210413745, 'reg_lambda': 0.0025488044873853287}. Best is trial 0 with value: 0.8383751144181651.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:05,377] Trial 1 finished with value: 0.8430404228304841 and parameters: {'num_leaves': 19, 'reg_alpha': 0.007223

[I 2020-09-21 19:06:16,409] Trial 16 finished with value: 0.8540688575899843 and parameters: {'num_leaves': 135, 'reg_alpha': 2.394640961734748, 'reg_lambda': 0.053698555204126495}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:16,972] Trial 17 finished with value: 0.8430773319160245 and parameters: {'num_leaves': 74, 'reg_alpha': 1.1618150173431718, 'reg_lambda': 2.260239220687913}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:1

[I 2020-09-21 19:06:25,948] Trial 33 finished with value: 0.8587784569049517 and parameters: {'num_leaves': 68, 'reg_alpha': 5.566511081473738, 'reg_lambda': 0.23522333814414323}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:26,446] Trial 34 finished with value: 0.8446570407771578 and parameters: {'num_leaves': 32, 'reg_alpha': 1.5667570262011166, 'reg_lambda': 0.050782693196799504}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:

[I 2020-09-21 19:06:35,691] Trial 50 finished with value: 0.8430699500989164 and parameters: {'num_leaves': 57, 'reg_alpha': 0.04375164264869748, 'reg_lambda': 0.47241406497358784}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:36,073] Trial 51 finished with value: 0.8635028198541352 and parameters: {'num_leaves': 36, 'reg_alpha': 6.415930891924241, 'reg_lambda': 1.0796157152099368}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:3

[I 2020-09-21 19:06:43,235] Trial 67 finished with value: 0.8509315853190422 and parameters: {'num_leaves': 54, 'reg_alpha': 2.005558664949153, 'reg_lambda': 2.849158432942903}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:43,667] Trial 68 finished with value: 0.8634954380370273 and parameters: {'num_leaves': 32, 'reg_alpha': 4.561538899025011, 'reg_lambda': 0.41082725621851596}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:44,0

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


[I 2020-09-21 19:06:44,543] Trial 70 finished with value: 0.8556411846340094 and parameters: {'num_leaves': 31, 'reg_alpha': 2.934332204756724, 'reg_lambda': 1.5409207957617306}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:44,956] Trial 71 finished with value: 0.8634880562199191 and parameters: {'num_leaves': 33, 'reg_alpha': 6.14491588768861, 'reg_lambda': 0.1846028861765985}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:45,37

[I 2020-09-21 19:06:53,250] Trial 87 finished with value: 0.8556559482682257 and parameters: {'num_leaves': 16, 'reg_alpha': 3.230111632006935, 'reg_lambda': 0.5306993926161488}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:53,900] Trial 88 finished with value: 0.8430625682818081 and parameters: {'num_leaves': 147, 'reg_alpha': 1.1670159625463137, 'reg_lambda': 1.2184644234662159}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:54

最終トライアル回数:100
ベストトライアル:
値:0.8681976555348866
パラメータ:
num_leaves:21
reg_alpha:3.917236543553505
reg_lambda:0.3417223854757551


In [74]:
# 使用する特徴量とターゲット
feature = train_dummied
target = train_target

X_train, X_test, y_train, y_test = train_test_split(feature, target, random_state=0, stratify=target)
lgb_params = {'objective': 'binary',
              'metric': 'auc', #コンペ指定の評価指標がAUC
              'verbosity': -1
             }

lgb_train = lgb.Dataset(X_train, y_train) #学習用
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train) #Boosting用
model = lgb.train(lgb_params, lgb_train, valid_sets=lgb_eval,
                  verbose_eval=50,  # 50イテレーション毎に学習結果出力
                  num_boost_round=1000,  # 最大イテレーション回数指定
                  early_stopping_rounds=100)

[I 2020-09-21 19:41:04,423] A new study created in memory with name: no-name-ddd019af-45f4-4dc9-afae-bfdd2a56dbdd

feature_fraction, val_score: -inf:   0%|                                                         | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds
[50]	valid_0's auc: 0.939607
[100]	valid_0's auc: 0.940678
[150]	valid_0's auc: 0.937823
Early stopping, best iteration is:
[62]	valid_0's auc: 0.944246



feature_fraction, val_score: 0.944246:  14%|######4                                      | 1/7 [00:00<00:01,  5.60it/s][I 2020-09-21 19:41:04,611] Trial 0 finished with value: 0.9442462087421944 and parameters: {'feature_fraction': 0.8999999999999999}. Best is trial 0 with value: 0.9442462087421944.

feature_fraction, val_score: 0.944246:  14%|######4                                      | 1/7 [00:00<00:01,  5.60it/s]

Training until validation scores don't improve for 100 rounds
[50]	valid_0's auc: 0.941927
[100]	valid_0's auc: 0.940054
Early stopping, best iteration is:
[43]	valid_0's auc: 0.945584



feature_fraction, val_score: 0.945584:  29%|############8                                | 2/7 [00:00<00:00,  5.96it/s][I 2020-09-21 19:41:04,755] Trial 1 finished with value: 0.9455842997323818 and parameters: {'feature_fraction': 0.7}. Best is trial 1 with value: 0.9455842997323818.

feature_fraction, val_score: 0.945584:  29%|############8                                | 2/7 [00:00<00:00,  5.96it/s]

...


min_data_in_leaf, val_score: 0.949866:  60%|###########################                  | 3/5 [00:00<00:00,  8.11it/s][I 2020-09-21 19:41:11,590] Trial 65 finished with value: 0.9385370205173952 and parameters: {'min_child_samples': 100}. Best is trial 64 with value: 0.9498661909009812.

min_data_in_leaf, val_score: 0.949866:  60%|###########################                  | 3/5 [00:00<00:00,  8.11it/s]

Training until validation scores don't improve for 100 rounds
[50]	valid_0's auc: 0.933363
[100]	valid_0's auc: 0.933541
Early stopping, best iteration is:
[6]	valid_0's auc: 0.941347



min_data_in_leaf, val_score: 0.949866:  60%|###########################                  | 3/5 [00:00<00:00,  8.11it/s][I 2020-09-21 19:41:11,659] Trial 66 finished with value: 0.9413470115967886 and parameters: {'min_child_samples': 50}. Best is trial 64 with value: 0.9498661909009812.

min_data_in_leaf, val_score: 0.949866:  80%|####################################         | 4/5 [00:00<00:00,  8.11it/s]

Training until validation scores don't improve for 100 rounds
[50]	valid_0's auc: 0.937556
[100]	valid_0's auc: 0.940856
Early stopping, best iteration is:
[12]	valid_0's auc: 0.942194



min_data_in_leaf, val_score: 0.949866: 100%|#############################################| 5/5 [00:00<00:00, 10.74it/s]


In [76]:
pred = model.predict(test_dummied) #口座申込み確率値

In [78]:
pd.DataFrame(np.round(pred)).to_csv('lightgbm_dummied_2020_09_21_19_41.csv', header=None)

In [77]:
pred

array([0.26635778, 0.3804145 , 0.26073896, 0.320003  , 0.29691831,
       0.29630557, 0.66584944, 0.46580113, 0.53586775, 0.36191902,
       0.24834713, 0.29246708, 0.44489293, 0.70068185, 0.24048095,
       0.38804845, 0.61781161, 0.30319431, 0.256928  , 0.24343333,
       0.52942036, 0.2579057 , 0.34938493, 0.52443487, 0.45468689,
       0.32542837, 0.34664369, 0.68441431, 0.50298342, 0.66338099,
       0.53042048, 0.68111918, 0.31419053, 0.4152301 , 0.62438985,
       0.67968274, 0.25290207, 0.37186938, 0.24834713, 0.60415672,
       0.62274191, 0.43104329, 0.51038037, 0.51797828, 0.49006949,
       0.41685699, 0.24048095, 0.59410794, 0.40083935, 0.55468969,
       0.451161  , 0.48055902, 0.68344803, 0.34145379, 0.52134593,
       0.24048095, 0.48989311, 0.38984695, 0.67654156, 0.34777955,
       0.4654982 , 0.26353928, 0.70068185, 0.31227701, 0.30409872,
       0.28175934, 0.5739748 , 0.58103276, 0.43435276, 0.36072735,
       0.65931537, 0.38034986, 0.44074669, 0.44421989, 0.33460

# Optuna用関数

In [97]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

def objective(trial):
    param = {
        'objective': 'cross_entropy',
        'metric': 'auc',
        'boosting': 'gbdt',
        'learning_rate': 0.05,
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'seed': 0,
        'verbosity': -1,
    }
    gbm = lgb.train(param, lgb_train, valid_sets=lgb_eval,
                    verbose_eval=False, num_boost_round=1000, early_stopping_rounds=100)
    y_prob = gbm.predict(X_test)
    y_pred = np.round(y_prob)
    return roc_auc_score(
        np.round(y_test.values),
        np.round(y_pred)
    )

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2020-09-22 11:01:23,684] A new study created in memory with name: no-name-f286c89e-c9fc-4ebf-b71d-22b1a824ab6d
[I 2020-09-22 11:01:23,687] A new study created in memory with name: no-name-a01e7e78-89bf-4c54-a8d8-4f40c4baea2d
feature_fraction, val_score: -inf:   0%|                                                         | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=35, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=35


num_leaves, val_score: 0.943622: 100%|#################################################| 20/20 [00:01<00:00, 10.62it/s]


regularization_factors, val_score: 0.949242:  55%|####################3                | 11/20 [00:00<00:00, 11.97it/s][I 2020-09-22 11:01:28,527] Trial 50 finished with value: 0.9475468331846566 and parameters: {'lambda_l1': 6.236583071098056e-06, 'lambda_l2': 0.07343157274129515}. Best is trial 47 with value: 0.9488849241748439.


min_data_in_leaf, val_score: 0.949242: 100%|#############################################| 5/5 [00:00<00:00, 10.79it/s]
[I 2020-09-22 11:01:29,683] Trial 0 finished with value: 0.8786797502230151 and parameters: {'lambda_l1': 0.001380063036828051, 'lambda_l2': 0.5486419227642434, 'num_leaves': 53, 'feature_fraction': 0.9631460604980978, 'bagging_fraction': 0.8505861168485154, 'bagging_freq': 9, 'min_child_samples': 35}. Best is trial 0 with value: 0.8786797502230151.
[I 2020-09-22 11:01:29,687] A new study created in memory with name: no-name-0f08d2b8-f6f3-43c2-9127-3a67dcb6396f
feature_fraction, val_score: -inf:   0%|                                                         | 0/7 [00:00<?, ?it/s]

...

[LightGBM] [Warning] min_data_in_leaf is set=35, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=35


num_leaves, val_score: 0.947458: 100%|#################################################| 20/20 [00:02<00:00,  9.35it/s]


regularization_factors, val_score: 0.951204:  50%|##################5                  | 10/20 [00:00<00:00, 11.27it/s][I 2020-09-22 11:07:31,376] Trial 50 finished with value: 0.9504014272970562 and parameters: {'lambda_l1': 1.1283964910570527e-06, 'lambda_l2': 5.504137017666187e-07}. Best is trial 44 with value: 0.9504014272970562.


min_data_in_leaf, val_score: 0.951204: 100%|#############################################| 5/5 [00:00<00:00, 11.17it/s]
[I 2020-09-22 11:07:32,617] Trial 49 finished with value: 0.8628902765388046 and parameters: {'lambda_l1': 0.08341816010842959, 'lambda_l2': 0.0002488543931371672, 'num_leaves': 289, 'feature_fraction': 0.8971599621299987, 'bagging_fraction': 0.6778194614820953, 'bagging_freq': 2, 'min_child_samples': 21}. Best is trial 46 with value: 0.8871543264942016.


Number of finished trials: 50
Best trial: {'lambda_l1': 0.2308143373205228, 'lambda_l2': 0.00035969557129608774, 'num_leaves': 174, 'feature_fraction': 0.8158668780442612, 'bagging_fraction': 0.7294074421813576, 'bagging_freq': 3, 'min_child_samples': 22}


In [116]:
study.best_params
gbm = lgb.train(study.best_params, lgb_train, valid_sets=lgb_eval)

[I 2020-09-22 11:19:09,170] A new study created in memory with name: no-name-483fc613-6667-4261-869a-6063f71dde0a

feature_fraction, val_score: inf:   0%|                                                          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=35, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=35
[LightGBM] [Warning] min_data_in_leaf is set=35, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=35
[LightGBM] [Warning] min_data_in_leaf is set=22, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=22
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000240 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1747
[LightGBM] [Info] Number of data points in the train set: 637, number of used features: 10
[LightGBM] [Info] Start training from score 0.444270
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[W 2020-09-22 11:19:12,401] Trial 0 failed because of the following error: KeyError('binary_logloss')
Traceback (most recent call last):
  File "C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\optuna\study.py", line 778, in _run_trial
    result = func(trial)
  File "C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\optuna\integration\_lightgbm_tuner\optimize.py", line 237, in __call__
    val_score = self._get_booster_best_score(booster)
  File "C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\optuna\integration\_lightgbm_tuner\optimize.py", line 114, in _get_booster_best_score
    val_score = booster.best_score[valid_name][metric]
KeyError: 'binary_logloss'


KeyError: 'binary_logloss'

In [120]:
study.best_params

{'lambda_l1': 0.2308143373205228,
 'lambda_l2': 0.00035969557129608774,
 'num_leaves': 174,
 'feature_fraction': 0.8158668780442612,
 'bagging_fraction': 0.7294074421813576,
 'bagging_freq': 3,
 'min_child_samples': 22}

In [128]:
# 使用する特徴量とターゲット
feature = train_quadratic
target = train_target

X_train, X_test, y_train, y_test = train_test_split(feature, target, random_state=0, stratify=target)

model = XGBClassifier(random_state=0)
model.fit(X_train, y_train)
model.score(X_test, y_test)

C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.8497652582159625

In [121]:
from xgboost import XGBRegressor, XGBClassifier
from sklearn.model_selection import GridSearchCV

params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

xgb = GridSearchCV(estimator=XGBClassifier(),
                   param_grid=params,
                   n_jobs=-1, cv=5)
xgb.fit(train_dummied, train_target)
model = xgb.best_estimator_

C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [122]:
model.score(X_test, y_test)

0.9577464788732394

In [124]:
pd.DataFrame(model.predict(test_dummied)).to_csv('xgboost_grid.csv', header=None)

In [168]:
feature = train_dummied
target = train_target

# feature['Gender_Male'][feature['Gender_Male'] == 0] = 'female'
# feature['Gender_Male'][feature['Gender_Male'] == 1] = 'male'


categorical_feature = ['Gender_Male']

X_train, X_test, y_train, y_test = train_test_split(feature, target, random_state=0, stratify=target)

lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=['Gender_Male'], free_raw_data=False)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train, categorical_feature=['Gender_Male'], free_raw_data=False)

# LightGBM のハイパーパラメータ
lgbm_params = {
    # 二値分類問題
    'objective': 'binary',
    # AUC の最大化を目指す
    'metric': 'auc',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

# 上記のパラメータでモデルを学習する
model = lgb.train(lgbm_params, lgb_train, valid_sets=[lgb_train,lgb_eval],
                  verbose_eval=50,  # 50イテレーション毎に学習結果出力
                  num_boost_round=1000,  # 最大イテレーション回数指定
                  early_stopping_rounds=100
                 )
# y_pred = model.predict(test_dummied, num_iteration=model.best_iteration)

[I 2020-09-22 12:12:44,631] A new study created in memory with name: no-name-69ce6ba9-1dff-4175-933b-31588f874ba0






  0%|                                                                                            | 0/7 [00:00<?, ?it/s]





feature_fraction, val_score: -inf:   0%|                                                         | 0/7 [00:00<?, ?it/s]C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[LightGBM] [Info] Number of positive: 283, number of negative: 354
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000228 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1748
[LightGBM] [Info] Number of data points in the train set: 637, number of used features: 10


C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.444270 -> initscore=-0.223850
[LightGBM] [Info] Start training from score -0.223850
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Info] Number of positive: 283, number of negative: 354
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000203 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1748
[LightGBM] [Info] Number of data points in the train set: 637, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.444270 -> initscore=-0.223850
[LightGBM] [Info] Start training from score -0.223850
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[10]	training's auc: 0.981244	valid_1's auc: 0.946476








min_data_in_leaf, val_score: 0.954148:  60%|###########################                  | 3/5 [00:01<00:00,  2.48it/s]





min_data_in_leaf, val_score: 0.954148:  80%|####################################         | 4/5 [00:01<00:00,  2.40it/s][I 2020-09-22 12:13:17,596] Trial 66 finished with value: 0.9464763603925067 and parameters: {'min_child_samples': 10}. Best is trial 64 with value: 0.9509366636931311.






min_data_in_leaf, val_score: 0.954148:  80%|####################################         | 4/5 [00:01<00:00,  2.40it/s]

[LightGBM] [Info] Number of positive: 283, number of negative: 354
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000205 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1748
[LightGBM] [Info] Number of data points in the train set: 637, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.444270 -> initscore=-0.223850
[LightGBM] [Info] Start training from score -0.223850
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [159]:
feature['Gender_Male'][feature['Gender_Male'] == 0] = 'female'

In [162]:
train_dummied

,Age,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,Gender_Male
0,59,0.786886,0.150498,220.178691,13.467617,21.729246,6.815731,3.112276,1.006802,male
1,69,1.003987,0.195625,221.218413,51.033462,64.752323,6.889608,3.051521,0.751346,male
2,65,0.657140,0.081287,320.770533,12.625011,30.613180,5.947767,2.489167,0.774952,male
3,65,0.906822,0.214173,369.278160,34.347597,54.510085,6.967183,3.613837,0.988155,male
4,22,1.734959,0.197706,222.782025,20.572891,170.010177,5.837537,3.068697,1.026654,female
...,...,...,...,...,...,...,...,...,...,...
845,25,0.767450,0.186467,215.656126,13.963269,20.160997,6.824975,3.622443,1.297731,male
846,57,0.867946,0.156997,213.810275,14.564305,20.122535,7.509893,3.619143,1.296843,male
847,50,0.826785,0.138898,230.340081,40.495102,49.620857,7.529840,3.724408,0.987033,male
848,32,0.761972,0.137969,214.170393,18.048321,54.447774,5.944855,3.150978,1.220787,female


In [137]:
pd.DataFrame(np.rint(model.predict(test_quadratic, num_iteration=model.best_iteration))).to_csv('auc094.csv', header=None)